In [80]:
##Install Lib
# !pip install pandas
# !pip install geodesic
# !pip install folium

import sys
import numpy as np
import pandas as pd
import folium
from folium import Circle
from geopy.distance import geodesic


In [81]:
def cal_distance_between_2_points(gdf, index1, index2):
    """
    cal_distance_between_2_points
    """
    coords_1 = (gdf.loc[index1,"lat"], gdf.loc[index1,"lng"])
    coords_2 = (gdf.loc[index2,"lat"],  gdf.loc[index2,"lng"])
    distance = geodesic(coords_1, coords_2).meters
    return distance

def cal_distance(df_data_locations):
    distances = []
    for i in range(len(df_data_locations)):
        distance = 0
        for j in range(len(df_data_locations)):
            distance += cal_distance_between_2_points(df_data_locations,i,j)
        distances.append(distance)

    df_data_locations['distances_total (m)'] = distances
    df_data_locations['distances_avg (m)'] = df_data_locations['distances_total (m)']/(len(df_data_locations)-1)

    return df_data_locations

def visualize_Circle(map, df_data, color, radius):
    str_popup = "{}, {}"
    for i in range(0,len(df_data_boundaries)):
        Circle(
            location=[df_data.iloc[i]['lat'], df_data.iloc[i]['lng']],
            radius=radius,
            color=color,
            popup=str_popup.format(df_data.iloc[i]['lat'], df_data.iloc[i]['lng'])).add_to(map)

def visualize_Marker(map, df_data, color):
    str_popup = "{}, {}"
    for id,row in df_data.iterrows():
        folium.Marker(location=[row['lat'],row['lng']],
                      popup=str_popup.format(row['lat'],row['lng']),
                      icon = folium.Icon(color=color)).add_to(map)

def visualize_on_map(df_data_boundaries, df_data_locations, zoom_start, df_data_key_points = pd.DataFrame(), df_data_excludes_points = pd.DataFrame()):
    """
    Visualize df_data_boundaries & df_data_locations
    :param df_data_boundaries:
    :param df_data_locations:
    :return:
    """
    # create map
    m = folium.Map(location=[df_data_locations.loc[0,"lat"], df_data_locations.loc[0,"lng"]], zoom_start=zoom_start)
    # boundaries map
    visualize_Circle(m, df_data_boundaries, 'red', 15)
    # location map
    visualize_Marker(m, df_data_locations, 'cadetblue')

    visualize_Marker(m, df_data_key_points, 'green')
    visualize_Marker(m, df_data_excludes_points, 'pink')


    return m

def convert_raw_data_to_df(raw_data_boundaries, raw_data_locations, raw_data_key_points = "" , raw_data_excludes_points= "" ):
    arr_data_locations = raw_data_locations.replace('\n',' ').replace(',',' ')
    arr_data_boundaries = raw_data_boundaries.replace('\n',' ').replace(',',' ')
    arr_data_key_points = raw_data_key_points.replace('\n',' ').replace(',',' ')
    arr_data_excludes_points = raw_data_excludes_points.replace('\n',' ').replace(',',' ')


    arr_data_locations = np.reshape(arr_data_locations.split(), (-1, 2))
    arr_data_boundaries = np.reshape(arr_data_boundaries.split(), (-1, 2))
    arr_data_key_points = np.reshape(arr_data_key_points.split(), (-1, 2))
    arr_data_excludes_points = np.reshape(arr_data_excludes_points.split(), (-1, 2))


    # df_data_locations.head()
    df_data_locations = pd.DataFrame({'lat':arr_data_locations[:, 0], 'lng': arr_data_locations[:, 1]})
    df_data_boundaries = pd.DataFrame({'lat':arr_data_boundaries[:, 1],'lng': arr_data_boundaries[:, 0]})
    df_data_key_points = pd.DataFrame({'lat':arr_data_key_points[:, 0], 'lng': arr_data_key_points[:, 1]})
    df_data_excludes_points = pd.DataFrame({'lat':arr_data_excludes_points[:, 0], 'lng': arr_data_excludes_points[:, 1]})


    #convert type to float64
    df_data_locations.lat = df_data_locations.lat.astype('float64')
    df_data_locations.lng = df_data_locations.lng.astype('float64')
    df_data_boundaries.lat = df_data_boundaries.lat.astype('float64')
    df_data_boundaries.lng = df_data_boundaries.lng.astype('float64')
    df_data_key_points.lat = df_data_key_points.lat.astype('float64')
    df_data_key_points.lng = df_data_key_points.lng.astype('float64')
    df_data_excludes_points.lat = df_data_excludes_points.lat.astype('float64')
    df_data_excludes_points.lng = df_data_excludes_points.lng.astype('float64')

    return df_data_boundaries, df_data_locations, df_data_key_points, df_data_excludes_points



# I.
### SETUP DATA
###### 1. Input raw_data_locations : [lat, lng]
###### 2. Input raw_data_boundaries : [lng, lat]
###### ==> Output: [lat, lng]
### Calculate Distances

In [82]:
# Input data locations here
raw_data_locations = """
13.08965,109.31149
"""
# Input data boundaries here
raw_data_boundaries = """
109.313995361328,13.0908241271973 109.313606262207,13.0895719528198 109.31014251709,13.0886745452881 109.309761047363,13.0898809432983 109.305297851562,13.0885810852052 109.303588867188,13.0880908966065 109.302856445313,13.0904016494752 109.302009582519,13.0900774002076 109.300506591797,13.0927076339722 109.300605773926,13.0946235656739 109.299926757812,13.0983018875123 109.301986694336,13.0971632003784 109.310028076172,13.095106124878 109.309959411621,13.0967140197754 109.307197570801,13.0986518859864 109.306427001953,13.1002960205079 109.305541992188,13.1034202575685 109.306182861328,13.1043119430543 109.311485290527,13.0972061157227 109.313995361328,13.0908241271973
"""

###--------------------------------------------------------------------#%%

df_data_boundaries,df_data_locations, df_data_key_points, df_data_excludes_points = convert_raw_data_to_df(raw_data_boundaries, raw_data_locations)

## Cal_distance
cal_distance(df_data_locations)


,lat,lng,distances_total (m),distances_avg (m)
0,13.08965,109.31149,0.0,NaN


### Visualize on Map

In [83]:
visualize_on_map(df_data_boundaries, df_data_locations, 13)

# II.
Extend

In [84]:
# Input data key_points here
raw_data_key_points = """
13.089770,109.311380 13.089650,109.311490 13.089740,109.311460 13.089380,109.311580
"""
# Input data excludes_points here
raw_data_excludes_points = """
13.094160,109.311220 13.092600,109.307570 13.089110,109.311670 13.091300,109.305410 13.090900,109.311480 13.090800,109.311560
"""
df_data_boundaries,df_data_locations, df_data_key_points, df_data_excludes_points = convert_raw_data_to_df(raw_data_boundaries, raw_data_locations, raw_data_key_points, raw_data_excludes_points)
visualize_on_map(df_data_boundaries, df_data_locations, 13, df_data_key_points,df_data_excludes_points)

-----
*Đồ đệ anh Vũ, không phải đệ chị Hồng*<br>
**#TASS**


